# Xarray Genetics API Prototype 

In [1]:
# Set to enable auto-complete for subclass properties
# See: https://github.com/ipython/ipython/issues/11653#issuecomment-492578777
%config Completer.use_jedi = False
import sys
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
sys.path.append(".")
from lib import api
xr.set_options(display_style='html')

## Data Structures

Data structure classes are all Xarray subclasses and do not maintain any state that will interfere with serialization or other Xarray functionality.  They function solely to:

- Provide casting mechanisms from one type to another (e.g. array containing allele index by chromosome to array containing alt allele counts)
- Apply preconditions on underlying arrays (e.g. count arrays have to be some unsigned integer type, all others will raise an error)
  - This allows for the data model to do most of the upfront type, value, and shape checking that many functions might otherwise do with a bunch of biolerplate.  In other words, this shifts data structure validation to the type hint system rather than "utility" functions.
- Allow pass-through support for duck array typing, meaning that this library should make as few assumptions as possible about what underlying array libraries are used (e.g. numpy, dask, xarray).  Anything implementing [\_\_array_function__](https://numpy.org/neps/nep-0018-array-function-protocol.html) is fine.
- Serve as anchors for documentation.

Any information that is specific to a genetics dataset needs to be represented in xarray Attributes, as coordinates, or as data variables in a Dataset -- no instance attributes should be attached to the class instances directly.  An example of this is phasing.  Whether or not calls are phased can vary within a single dataset but this should not be represented as say, an attribute on a ```GenotypeIndexArray``` but rather as a separate data variable in an ```xarray.Dataset```.  Examples of this are shown later.

### Creating Arrays

In [3]:
# 10 variants, 5 samples, and 2 chromosomes for which hard calls are simulated
shape = (10, 5, 2)

In [17]:
# From numpy:
gt = np.random.randint(0, 2, size=shape, dtype=np.uint8) # Draw from [0, 1]
gt = api.GenotypeIndexArray(allele_index=gt, attrs={'description': 'Bi-allelic, diploid hard call array example [numpy]'})
gt

<xarray.GenotypeIndexArray (variant: 10, sample: 5, ploidy: 2)>
array([[[1, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [0, 0]],

       [[1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0]],

       [[0, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 1]],

       [[1, 0],
        [0, 1],
        [0, 0],
        [1, 0],
        [0, 1]],

       [[0, 0],
        [0, 1],
        [1, 1],
        [1, 0],
        [0, 1]],

       [[0, 0],
        [1, 0],
        [1, 0],
        [1, 1],
        [0, 1]],

       [[0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]],

       [[0, 0],
        [0, 1],
        [1, 1],
        [1, 0],
        [1, 0]],

       [[0, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1]],

       [[0, 0],
        [1, 1],
        [0, 0],
        [0, 1],
        [0, 0]]], dtype=uint8)
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
  * ploidy   (ploidy) int64 0 1

In [56]:
# From dask:
gt = da.random.randint(0, 2, size=shape, dtype=np.uint8)
gt = api.GenotypeIndexArray(allele_index=gt, attrs={'description': 'Bi-allelic, diploid hard call array example [dask]'})
gt

<xarray.GenotypeIndexArray 'randint-e6e1d3880b0a9a13afbc13570ddcba78' (variant: 10, sample: 5, ploidy: 2)>
dask.array<randint, shape=(10, 5, 2), dtype=uint8, chunksize=(10, 5, 2), chunktype=numpy.ndarray>
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
  * ploidy   (ploidy) int64 0 1
Attributes:
    description:  Bi-allelic, diploid hard call array example [dask]

In [57]:
gt.data

dask.array<randint, shape=(10, 5, 2), dtype=uint8, chunksize=(10, 5, 2), chunktype=numpy.ndarray>

In [58]:
# From xarray (gt is already an Xarray DataArray)
# Note however, that when creating a DataArray from another DataArray, Xarray
# will call .asarray and force the underlying data to numpy 
api.GenotypeIndexArray(gt)

<xarray.DataArray 'randint-e6e1d3880b0a9a13afbc13570ddcba78' (variant: 10, sample: 5, ploidy: 2)>
array([[[0, 1],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0]],

       [[0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 1]],

       [[0, 0],
        [0, 1],
        [1, 1],
        [1, 0],
        [1, 0]],

       [[1, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [0, 1],
        [0, 0],
        [0, 1]],

       [[0, 1],
        [1, 0],
        [0, 1],
        [0, 0],
        [1, 0]],

       [[0, 1],
        [1, 1],
        [0, 0],
        [1, 0],
        [0, 0]],

       [[1, 1],
        [1, 1],
        [0, 0],
        [1, 0],
        [1, 1]],

       [[0, 0],
        [0, 1],
        [0, 0],
        [0, 1],
        [0, 0]],

       [[1, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 1]]], dtype=uint8)
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
  * ploidy   (ploidy) int64 0 1
Attributes:
    description:  Bi-allelic, diploid hard call array example [dask]

### Creating Datasets

To represent more complex structures, such as CNV data for a multi-allelic, polyploid experiment, ```Dataset``` instances can be used.  These structures constitute the inputs to more advanced analyses:

In [59]:
# Create an array containing hard calls where values correspond to allele index (up to at most 3 alleles possible in this case)
gt_idx = api.GenotypeIndexArray(allele_index=np.random.randint(0, 3, size=shape, dtype=np.uint8))

# Create an array containing the copy numbers as counts of the alleles above on each chromosome
gt_cts = api.GenotypeCountArray(allele_count=np.random.randint(0, 5, size=shape, dtype=np.uint8))

# Combine the two arrays into a single dataset
cnv_ds = api.GenotypeAlleleCountDataset(allele_index=gt_idx, allele_count=gt_cts, attrs={'name': 'CNV Dataset Example'})
cnv_ds

<xarray.GenotypeAlleleCountDataset>
Dimensions:       (ploidy: 2, sample: 5, variant: 10)
Coordinates:
  * variant       (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample        (sample) int64 0 1 2 3 4
  * ploidy        (ploidy) int64 0 1
Data variables:
    allele_index  (variant, sample, ploidy) uint8 0 2 1 0 2 0 0 ... 1 0 2 1 1 0
    allele_count  (variant, sample, ploidy) uint8 2 1 0 3 4 2 1 ... 0 4 2 1 4 1
Attributes:
    name:     CNV Dataset Example

### Data Structure Conversions

The API facilitates conversions to different representations where possible through methods on the data structure subclasses.  These conversions often result in a loss of information (e.g. reductions across a dimension) -- i.e. they are not invertible.  This makes it easy for an analysis to start with a complex N-dimensional structure and call these conversions where needed as inputs to algorithms that generally expect simpler structures (e.g. LD estimation only needs alternate allele counts as a 2D array).

These examples show a few conversions to dosage arrays:

#### From Hard Calls

In [66]:
# Simulate an array of diploid hard calls
gt = np.random.randint(0, 2, size=shape, dtype=np.uint8) # Draw from [0, 1]
gt = api.GenotypeIndexArray(allele_index=gt, attrs={'description': 'Calculating dosage from hard calls example'})
gt

<xarray.GenotypeIndexArray (variant: 10, sample: 5, ploidy: 2)>
array([[[0, 0],
        [0, 1],
        [0, 1],
        [1, 1],
        [0, 1]],

       [[0, 0],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 0],
        [1, 1],
        [1, 0],
        [0, 0]],

       [[1, 0],
        [0, 1],
        [1, 0],
        [1, 1],
        [0, 0]],

       [[1, 1],
        [0, 1],
        [1, 0],
        [1, 1],
        [0, 0]],

       [[0, 0],
        [0, 1],
        [1, 1],
        [0, 1],
        [0, 0]],

       [[1, 0],
        [0, 1],
        [1, 1],
        [0, 0],
        [0, 1]],

       [[1, 1],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 1]],

       [[1, 0],
        [0, 1],
        [0, 1],
        [0, 0],
        [1, 0]],

       [[1, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 1]]], dtype=uint8)
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
  * ploidy   (ploidy) int64 0 1
Attributes:
    description:  Calculating dosage from hard calls example

In [67]:
# Convert to dosages (sum along ploidy dimension)
gtd = gt.to_dosage_array()
gtd

<xarray.GenotypeDosageArray (variant: 10, sample: 5)>
array([[0, 1, 1, 2, 1],
       [0, 2, 2, 2, 2],
       [2, 1, 2, 1, 0],
       [1, 1, 1, 2, 0],
       [2, 1, 1, 2, 0],
       [0, 1, 2, 1, 0],
       [1, 1, 2, 0, 1],
       [2, 0, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [2, 1, 1, 2, 2]], dtype=uint64)
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
Attributes:
    description:  Calculating dosage from hard calls example

In [68]:
# This is the usual input many genetics methods operate on
gtd.data

array([[0, 1, 1, 2, 1],
       [0, 2, 2, 2, 2],
       [2, 1, 2, 1, 0],
       [1, 1, 1, 2, 0],
       [2, 1, 1, 2, 0],
       [0, 1, 2, 1, 0],
       [1, 1, 2, 0, 1],
       [2, 0, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [2, 1, 1, 2, 2]], dtype=uint64)

In [69]:
# This is the same, but w/o API type preservation
gt.sum(dim='ploidy')

<xarray.DataArray (variant: 10, sample: 5)>
array([[0, 1, 1, 2, 1],
       [0, 2, 2, 2, 2],
       [2, 1, 2, 1, 0],
       [1, 1, 1, 2, 0],
       [2, 1, 1, 2, 0],
       [0, 1, 2, 1, 0],
       [1, 1, 2, 0, 1],
       [2, 0, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [2, 1, 1, 2, 2]], dtype=uint64)
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4

#### From Probabilities

If the genotype calls for an experiment are imputed (or probabilistic for some other reason), this is what converting to dosage looks like:

In [105]:
gp = np.random.rand(*shape, 2)
gp /= gp.sum(axis=-1, keepdims=True)
gp.shape

(10, 5, 2, 2)

In [106]:
gp[0, 0, 0]

array([0.46215599, 0.53784401])

In [107]:
gp = api.GenotypeProbabilityArray(allele_probability=gp, attrs={'description': 'Genotype probability example'})
gp

<xarray.GenotypeProbabilityArray (variant: 10, sample: 5, ploidy: 2, allele: 2)>
array([[[[0.46215599, 0.53784401],
         [0.6502239 , 0.3497761 ]],

        [[0.66684287, 0.33315713],
         [0.76283501, 0.23716499]],

        [[0.13560125, 0.86439875],
         [0.52437455, 0.47562545]],

        [[0.95725457, 0.04274543],
         [0.51411001, 0.48588999]],

        [[0.89597974, 0.10402026],
         [0.57456092, 0.42543908]]],


       [[[0.20257324, 0.79742676],
         [0.22423956, 0.77576044]],

        [[0.01352977, 0.98647023],
         [0.59333921, 0.40666079]],

        [[0.81736484, 0.18263516],
         [0.50438072, 0.49561928]],

        [[0.7762459 , 0.2237541 ],
         [0.37549534, 0.62450466]],

        [[0.4496279 , 0.5503721 ],
         [0.15842174, 0.84157826]]],


       [[[0.23878429, 0.76121571],
         [0.31762664, 0.68237336]],

        [[0.66078499, 0.33921501],
         [0.98495087, 0.01504913]],

        [[0.98783171, 0.01216829],
         [0.97072429, 0.02927571]],

        [[0.51026325, 0.48973675],
         [0.40280271, 0.59719729]],

        [[0.81829893, 0.18170107],
         [0.39353409, 0.60646591]]],


       [[[0.18167828, 0.81832172],
         [0.72023526, 0.27976474]],

        [[0.62750605, 0.37249395],
         [0.4967875 , 0.5032125 ]],

        [[0.40889419, 0.59110581],
         [0.89856159, 0.10143841]],

        [[0.21573685, 0.78426315],
         [0.81741825, 0.18258175]],

        [[0.88309737, 0.11690263],
         [0.42853406, 0.57146594]]],


       [[[0.32228545, 0.67771455],
         [0.44741407, 0.55258593]],

        [[0.99482099, 0.00517901],
         [0.94696443, 0.05303557]],

        [[0.64447021, 0.35552979],
         [0.31814942, 0.68185058]],

        [[0.29467924, 0.70532076],
         [0.56878016, 0.43121984]],

        [[0.62000962, 0.37999038],
         [0.52318117, 0.47681883]]],


       [[[0.37166202, 0.62833798],
         [0.64980066, 0.35019934]],

        [[0.36815873, 0.63184127],
         [0.5639367 , 0.4360633 ]],

        [[0.54825514, 0.45174486],
         [0.34958918, 0.65041082]],

        [[0.42173575, 0.57826425],
         [0.53531052, 0.46468948]],

        [[0.69093139, 0.30906861],
         [0.0398325 , 0.9601675 ]]],


       [[[0.92844375, 0.07155625],
         [0.73057233, 0.26942767]],

        [[0.29526761, 0.70473239],
         [0.42599666, 0.57400334]],

        [[0.18415768, 0.81584232],
         [0.5102366 , 0.4897634 ]],

        [[0.43860134, 0.56139866],
         [0.25408926, 0.74591074]],

        [[0.63235553, 0.36764447],
         [0.51428958, 0.48571042]]],


       [[[0.307179  , 0.692821  ],
         [0.68489503, 0.31510497]],

        [[0.80428817, 0.19571183],
         [0.79267303, 0.20732697]],

        [[0.64986248, 0.35013752],
         [0.9759913 , 0.0240087 ]],

        [[0.61901599, 0.38098401],
         [0.53008484, 0.46991516]],

        [[0.92923801, 0.07076199],
         [0.02181106, 0.97818894]]],


       [[[0.52524929, 0.47475071],
         [0.69446452, 0.30553548]],

        [[0.88493783, 0.11506217],
         [0.80166748, 0.19833252]],

        [[0.52869012, 0.47130988],
         [0.58912405, 0.41087595]],

        [[0.52131282, 0.47868718],
         [0.38504539, 0.61495461]],

        [[0.86322939, 0.13677061],
         [0.33719255, 0.66280745]]],


       [[[0.56311248, 0.43688752],
         [0.61745563, 0.38254437]],

        [[0.63079316, 0.36920684],
         [0.67069745, 0.32930255]],

        [[0.26009657, 0.73990343],
         [0.67696459, 0.32303541]],

        [[0.486271  , 0.513729  ],
         [0.53164359, 0.46835641]],

        [[0.45341459, 0.54658541],
         [0.6443261 , 0.3556739 ]]]])
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
  * ploidy   (ploidy) int64 0 1
  * allele   (allele) int64 0 1
Attributes:
    description:  Genotype probability example

In [111]:
# Dosages will be calculated as Pr(Heterozygous) + 2 * Pr(Homozygous Alternate) (Allele 0 assumed as reference)
gp.to_dosage_array()

<xarray.GenotypeDosageArray (variant: 10, sample: 5)>
array([[0.8876201 , 0.57032212, 1.34002419, 0.52863542, 0.52945933],
       [1.5731872 , 1.39313102, 0.67825444, 0.84825876, 1.39195036],
       [1.44358908, 0.35426414, 0.041444  , 1.08693404, 0.78816699],
       [1.09808647, 0.87570645, 0.69254422, 0.9668449 , 0.68836858],
       [1.23030048, 0.05821458, 1.03738037, 1.13654059, 0.85680921],
       [0.97853731, 1.06790457, 1.10215568, 1.04295373, 1.26923611],
       [0.34098393, 1.27873573, 1.30560572, 1.3073094 , 0.8533549 ],
       [1.00792597, 0.4030388 , 0.37414622, 0.85089917, 1.04895092],
       [0.78028618, 0.31339469, 0.88218583, 1.09364179, 0.79957806],
       [0.81943189, 0.69850939, 1.06293885, 0.98208541, 0.90225931]])
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 6 7 8 9
  * sample   (sample) int64 0 1 2 3 4
Attributes:
    description:  Genotype probability example

## Operations

TBD

-----

### Example Ideas

- handling missing values
- adding fields to dataset as joins
- copy number
- converting from calls to dosages/alt_counts
- are subtypes preserved when added to dataset?
- show matrix of convertability between types?
    
### Notes

- Xarray often invokes constructors on DataArray/Dataset like this: ```type(self)(*args, **kwargs)```
  - This means that subclasses must be able to differentiate constructor params
- Do any loci have more than 128 alleles (i.e. should np.uint8 be a safe assumption for allele dimension)?
- xr.DataArray(xr.DataArray()) will call .asarray on input argument pushing .data into memory as numpy